In [ ]:
!pip install feedparser # RSS에서 XML 태그별 정보 추출
!pip install newspaper3k #인터넷신문기사 분석 (Ariticle() 사용하기 위한 목적)
!pip install konlpy # 우리말 형태소 분석 (주어진 문장에서 명사만 추출할 목적)
!pip install lxml[html_clean]

import feedparser
from newspaper import Article
from konlpy.tag import Okt

from collections import Counter

import hashlib # 해시 키 만들기 용

# from bs4 import BeautifulSoup # 명사 추출 후 출현 빈도 확인용(TF 구현용)

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=1dc79848a3250ac80875cad590ce40dadd8252be5e719ec567727a4bdf63b3fc
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.3 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=507587e0d46d358bf02bfdf186bbde172d3813a5e71510cf8925f327c32eb409
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df4

In [ ]:
# 1. 모든 RSS 파일(xml 형식)을 돌아다니면서 기사의 제목/link를 추출
# urls에 우리가 검색할 RSS파일들의 주소를 list로
urls = ["https://rss.etnews.com/Section901.xml",
        "https://rss.etnews.com/Section902.xml",
        "https://rss.etnews.com/Section903.xml",
        "https://rss.etnews.com/Section904.xml",
        ]

# 아래 함수는 RSS 목록의 list인 urls를 받아서
# 해당 urls 내에 존재하는 title과 link를 추출
def crawl_rss(urls):
    array_rss = []  # title과 link가 들어갈 list
    seen_titles = set()  # 중복 검사를 위한 set

    for index_url in urls:  # urls 내의 url들을 순회
        print("[Crawl RSS]", index_url)
        parse_rss = feedparser.parse(index_url)  # 현재 xml 파싱 후 결과를 parse_rss에 저장

        for p in parse_rss.entries:  # parse 후 모든 entry 순회
            # title을 해시화한 후 중복 검사
            # 해시화 하는 건 그냥 평문보다는 이뻐서 했음
            title_hash = hashlib.sha256(p.title.encode()).hexdigest()
            if title_hash not in seen_titles:  # 중복되지 않은 경우만 추가
                seen_titles.add(title_hash)
                array_rss.append({'title': p.title, 'link': p.link})

    return array_rss


list_articls = crawl_rss(urls)
print(list_articls)

[Crawl RSS] https://rss.etnews.com/Section901.xml
[Crawl RSS] https://rss.etnews.com/Section902.xml
[Crawl RSS] https://rss.etnews.com/Section903.xml
[Crawl RSS] https://rss.etnews.com/Section904.xml
[{'title': "위고페어, '원클릭 위조상품 신고서비스' 출시", 'link': 'https://www.etnews.com/20241105000329'}, {'title': "쿠팡, 3분기 매출 10조6900억 '역대 최대'...고객도 11% 증가", 'link': 'https://www.etnews.com/20241106000001'}, {'title': "'장 담그기' 유네스코 인류무형유산으로… “밥·김치 등 韓 문화의 핵심”", 'link': 'https://www.etnews.com/20241105000378'}, {'title': '막오른 `미국의 선택`...해리스-트럼프 초접전', 'link': 'https://www.etnews.com/20241105000263'}, {'title': "日 “공주는 안돼”… 유엔 '여성 왕위 계승' 권고 사실상 거부", 'link': 'https://www.etnews.com/20241105000379'}, {'title': '축구장에 날벼락…페루 선수 8명 쓰러졌다', 'link': 'https://www.etnews.com/20241105000371'}, {'title': "美 뉴욕서 6명 목숨 앗아간 '지하철 서핑'… 과연 뭐길래?", 'link': 'https://www.etnews.com/20241105000253'}, {'title': '짐 켈러·조주완 CEO 회동…텐스토렌트·LG전자 투자 협력 논의', 'link': 'https://www.etnews.com/20241105000404'}, {'title': "'금투세 폐지', 이번달 처리 될까…

In [ ]:
# RSS에서 추출한 기사 list에서 본문 title과 text를 추출
# 아래의 함수는 하나의 url을 입력 받아, 그 안의 title과 text를  추출하게 됨.
def crawl_article(url, languege='ko'):
    print("[Crawl Article]", url) # 현재 url
    a = Article(url, languege = languege) # Ariticle에 url과 언어옵션 지정
    a.download() # 해당 url의 기사 다운로드
    a.parse() # parsing!
    return a.title, a.text

print(crawl_article(list_articls[0]['link']))

[Crawl Article] https://www.etnews.com/20241105000329
("위고페어, '원클릭 위조상품 신고서비스' 출시", "상품 URL만 올리면 전 세계 온라인마켓 위조품 신고 끝\n\n인공지능(AI) 기반 위조상품 모니터링 및 차단 플랫폼을 운영하는 위고페어가 URL만 입력하면 간편하게 위조상품 신고할 수 있는 '원클릭 위조상품 신고 서비스'를 출시했다고 6일 밝혔다.\n\n\n\n'원클릭 위조상품 신고 서비스'는 위조품 판매 페이지 URL 입력만으로 위조상품 신고가 완료되어 전문지식이 없어도 누구나 쉽게 이용할 수 있다. 일명 '가품' 또는 '짝퉁'이라고 불리는 위조상품이 판매되는 것을 알고 있어도 플랫폼별로 상이한 신고 절차, 언어 장벽, 복잡한 신고 프로세스 등으로 대응을 못해 어려움을 겪던 기업들이 보다 쉽게 브랜드를 보호할 수 있도록 설계됐다. 특히 건별 신고 및 결제가 가능해 기업의 비용 부담을 대폭 줄인 것이 특징이다.\n\n\n\n위고페어 '원클릭 위조상품 신고'서비스는 국내뿐 아니라 다양한 글로벌 쇼핑몰에서 우리 기업의 브랜드 보호를 지원한다. 네이버, 쿠팡 등을 비롯해 알리바바, 테무, 쉬인 등 중국의 주요 이커머스 플랫폼은 물론, 쇼피, 라자다와 같은 동남아 플랫폼, 아마존, 이베이 등 글로벌 플랫폼의 위조상품까지 모두 신고가 가능하다.\n\n\n\n변리사와 지식재산보호원 위조상품 단속팀 출신 전문가들이 직접 신고 프로세스를 관리해 99%의 위조상품 차단 실적을 자랑한다. 기업별 전담 매니저를 배정해 맞춤형 서비스를 신속하게 제공한다.\n\n\n\n김종면 위고페어 CEO는 “위조상품 유통은 기업의 브랜드 이미지 훼손뿐만 아니라 매출 손실, 일자리 감소, 세수 감소 등 국가 경제에도 심각한 피해를 주고 있다”라며 “위고페어는 최첨단 AI 기술과 글로벌 플랫폼 위조품 차단 실무 경험을 보유한 전문 인력들을 바탕으로 우리 기업들의 브랜드를 효과적으로 보호하겠다”고 강조했다.\n\n\n\n위고페어는 '원클릭 위조상품 

In [ ]:
for article in list_articls:
    _, text = crawl_article(article['link'])
    article['text'] = text

print(list_articls[0]['text'])

[Crawl Article] https://www.etnews.com/20241105000329
[Crawl Article] https://www.etnews.com/20241106000001
[Crawl Article] https://www.etnews.com/20241105000378
[Crawl Article] https://www.etnews.com/20241105000263
[Crawl Article] https://www.etnews.com/20241105000379
[Crawl Article] https://www.etnews.com/20241105000371
[Crawl Article] https://www.etnews.com/20241105000253
[Crawl Article] https://www.etnews.com/20241105000404
[Crawl Article] https://www.etnews.com/20241105000115
[Crawl Article] https://www.etnews.com/20241105000374
[Crawl Article] https://www.etnews.com/20241105000310
[Crawl Article] https://www.etnews.com/20241105000240
[Crawl Article] https://www.etnews.com/20241104000451
[Crawl Article] https://www.etnews.com/20241105000350
[Crawl Article] https://www.etnews.com/20241105000334
[Crawl Article] https://www.etnews.com/20241105000333
[Crawl Article] https://www.etnews.com/20241105000255
[Crawl Article] https://www.etnews.com/20241105000225
[Crawl Article] https://www.

In [ ]:
# 3. 긁어온 본문 test에서 명사 추출(키워드, 빈도수)
def get_keywords(text, nKeywords = 10):
    list_keywords = [] # 빈 키워드 리스트. 추후 append로 모든 기사에 대해 내용 추가

    spliter = Okt() # 문장을 형태소별로 쪼개는 spliter

    nouns = spliter.nouns(text) # noun을 추출하여 nouns에 저장
    count = Counter(nouns) # 각 명사의 수

    for n, c in count.most_common(nKeywords): # 가장 출현빈도 높은 명사부터 순차적으로 10번
        item = {'keyword' : n, 'count' : c} # 저장은 키워드, 카운트 형식으로
        list_keywords.append(item)

    return list_keywords

In [ ]:
print(get_keywords(list_articls[0]['text']))

[{'keyword': '상품', 'count': 14}, {'keyword': '신고', 'count': 13}, {'keyword': '위조', 'count': 13}, {'keyword': '위고', 'count': 7}, {'keyword': '페어', 'count': 7}, {'keyword': '서비스', 'count': 7}, {'keyword': '플랫폼', 'count': 6}, {'keyword': '기업', 'count': 6}, {'keyword': '수', 'count': 5}, {'keyword': '등', 'count': 5}]


In [ ]:
for article in list_articls:
    keywords = get_keywords(article['text'])
    article['keywords'] = keywords

print(list_articls[0])

{'title': "위고페어, '원클릭 위조상품 신고서비스' 출시", 'link': 'https://www.etnews.com/20241105000329', 'text': "상품 URL만 올리면 전 세계 온라인마켓 위조품 신고 끝\n\n인공지능(AI) 기반 위조상품 모니터링 및 차단 플랫폼을 운영하는 위고페어가 URL만 입력하면 간편하게 위조상품 신고할 수 있는 '원클릭 위조상품 신고 서비스'를 출시했다고 6일 밝혔다.\n\n\n\n'원클릭 위조상품 신고 서비스'는 위조품 판매 페이지 URL 입력만으로 위조상품 신고가 완료되어 전문지식이 없어도 누구나 쉽게 이용할 수 있다. 일명 '가품' 또는 '짝퉁'이라고 불리는 위조상품이 판매되는 것을 알고 있어도 플랫폼별로 상이한 신고 절차, 언어 장벽, 복잡한 신고 프로세스 등으로 대응을 못해 어려움을 겪던 기업들이 보다 쉽게 브랜드를 보호할 수 있도록 설계됐다. 특히 건별 신고 및 결제가 가능해 기업의 비용 부담을 대폭 줄인 것이 특징이다.\n\n\n\n위고페어 '원클릭 위조상품 신고'서비스는 국내뿐 아니라 다양한 글로벌 쇼핑몰에서 우리 기업의 브랜드 보호를 지원한다. 네이버, 쿠팡 등을 비롯해 알리바바, 테무, 쉬인 등 중국의 주요 이커머스 플랫폼은 물론, 쇼피, 라자다와 같은 동남아 플랫폼, 아마존, 이베이 등 글로벌 플랫폼의 위조상품까지 모두 신고가 가능하다.\n\n\n\n변리사와 지식재산보호원 위조상품 단속팀 출신 전문가들이 직접 신고 프로세스를 관리해 99%의 위조상품 차단 실적을 자랑한다. 기업별 전담 매니저를 배정해 맞춤형 서비스를 신속하게 제공한다.\n\n\n\n김종면 위고페어 CEO는 “위조상품 유통은 기업의 브랜드 이미지 훼손뿐만 아니라 매출 손실, 일자리 감소, 세수 감소 등 국가 경제에도 심각한 피해를 주고 있다”라며 “위고페어는 최첨단 AI 기술과 글로벌 플랫폼 위조품 차단 실무 경험을 보유한 전문 인력들을 바탕으로 우리 기업들의 브랜드를 효과적으로 보호하겠다”고 강조했다.\n\n\n\n위고페어

In [ ]:
# 검색어를 입력받아서, 해당 검색어를 가진 문서를 출력
query = input()

# 아래 함수는 쿼리를 입력받은 후, 정해진 문서의 키워드 리스트에서
# 쿼리가 그 키워드 중 하나로 존재라는지 검색
def search_articles(query, list_keywords):
    nWords = 0 # 쿼리의 출현 횟수
    for kw in list_keywords:
        if query == kw['keyword']: # 키워드가 있다면
            nWords = kw['count']
            break

    return nWords

results = []

# 문서 전체 검색
for article in list_articls:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        results.append({
            'TF': nQuery, # 검색 키워드 출현 횟수
            'Title': article['title'], # 검색된 기사의 제목
            'URL': article['link'] # 검색된 기사의 URL
        })

# 출현 횟수로 정렬 (내림차순)
results.sort(key=lambda x: x['TF'], reverse=True)

# 정렬된 결과 출력
for result in results:
    print('[TF]', result['TF'], '[Title]', result['Title'])
    print('[URL]', result['URL'])

미국
[TF] 8 [Title] 막오른 `미국의 선택`...해리스-트럼프 초접전
[URL] https://www.etnews.com/20241105000263
[TF] 7 [Title] 외산 장비 설 곳 잃어가는 中 반도체 시장
[URL] https://www.etnews.com/20241105000240
[TF] 7 [Title] [2024 美 대선] 선택의 날 밝았다…해리스·트럼프 막판까지 초접전
[URL] https://www.etnews.com/20241104000315
[TF] 4 [Title] 韓·美, 원자력수출협력 합의…산업부 “제2 웨스팅하우스 분쟁 막는다”
[URL] https://www.etnews.com/20241105000334
[TF] 3 [Title] 美 뉴욕서 6명 목숨 앗아간 '지하철 서핑'… 과연 뭐길래?
[URL] https://www.etnews.com/20241105000253
